# Testing your environment

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.scatter('A', 'B', data=pd.DataFrame(np.random.rand(10, 2), columns=['A', 'B']))

# Data download

In [ ]:
from tools import download_file, unzip, ResourceMonitor
mon = ResourceMonitor()
mon.start()
mon.tag('download')
url = 'URL WILL BE UPDATED LATER'
file = download_file(url, fname='synig.zip')
mon.tag('unzip')
unzip(file)
mon.tag('done')
mon.stop()
mon.plot()